<a href="https://colab.research.google.com/github/SWazniewicz/UMwF-1-/blob/main/Zadanie_3(2)_UMwF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Stwórz model generujący sygnały kupna i sprzedaży na rynku
Decyzje ma generować jeden z algorytmów uczenia maszynowego, po optymalizacji hiperparametrów
Zrób backtest wybranego modelu, użyj biblioteki przystosowanej do backtestów
Pamiętaj o wnioskach i wizualizacji wyników
4 pkt z oceny będą zależne od wyników inwestycji
Inwestujesz w wylosowaną spółkę przez określony okres, ale można wykorzystać także inne dane niż historyczne (np. obliczone wskaźniki)

Strategia inwestycyjna (decyzje wejścia i wyjścia) dla spółki  Microsoft (MSFT), test w okresie od 01.01.2024 - 06.05.2024


In [3]:
!pip install yfinance backtesting scikit-learn ta --quiet

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.1/192.1 kB 12.1 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import yfinance as yf
import ta

from backtesting import Backtest, Strategy
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import HistGradientBoostingClassifier
from scipy.stats import randint, uniform

/usr/local/lib/python3.12/dist-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [5]:
symbol = "MSFT"
start_train = "2020-01-01"
end_all = "2024-05-06"
test_start = "2024-01-01"
test_end = "2024-05-06"

data = yf.download(symbol, start=start_train, end=end_all)
data.columns = data.columns.get_level_values(0)
data.head()

/tmp/ipython-input-3289600845.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(symbol, start=start_train, end=end_all)
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Date,,,,,
2020-01-02,152.505692,152.610135,150.331386,150.758649,22622100
2020-01-03,150.606720,151.869532,150.075013,150.321887,21116200
2020-01-06,150.996017,151.062488,148.603320,149.144532,20813700
2020-01-07,149.619232,151.603644,149.372372,151.271334,21634100
2020-01-08,152.002472,152.676610,149.970582,150.901070,27746500


In [8]:
print(data.columns)
print(data.head())

Index(['Close', 'High', 'Low', 'Open', 'Volume'], dtype='object', name='Price')
Price            Close        High         Low        Open    Volume
Date                                                                
2020-01-02  152.505692  152.610135  150.331386  150.758649  22622100
2020-01-03  150.606720  151.869532  150.075013  150.321887  21116200
2020-01-06  150.996017  151.062488  148.603320  149.144532  20813700
2020-01-07  149.619232  151.603644  149.372372  151.271334  21634100
2020-01-08  152.002472  152.676610  149.970582  150.901070  27746500


In [9]:
df = data.copy()

df["return_1d"] = df["Close"].pct_change()
df["return_3d"] = df["Close"].pct_change(3)
df["return_5d"] = df["Close"].pct_change(5)
df["return_10d"] = df["Close"].pct_change(10)

df["sma_10"] = df["Close"].rolling(10).mean()
df["sma_20"] = df["Close"].rolling(20).mean()
df["sma_50"] = df["Close"].rolling(50).mean()
df["ema_20"] = df["Close"].ewm(span=20).mean()

df["atr_14"] = ta.volatility.average_true_range(df["High"], df["Low"], df["Close"], window=14)
df["bb_high"] = ta.volatility.BollingerBands(df["Close"], window=20).bollinger_hband()
df["bb_low"] = ta.volatility.BollingerBands(df["Close"], window=20).bollinger_lband()
df["bb_width"] = (df["bb_high"] - df["bb_low"]) / df["Close"]

df["rsi_14"] = ta.momentum.rsi(df["Close"], window=14)
df["stoch_k"] = ta.momentum.stoch(df["High"], df["Low"], df["Close"], window=14, smooth_window=3)
df["adx"] = ta.trend.adx(df["High"], df["Low"], df["Close"], window=14)

macd = ta.trend.MACD(df["Close"])
df["macd"] = macd.macd()
df["macd_signal"] = macd.macd_signal()
df["macd_diff"] = macd.macd_diff()

df["vol_sma_10"] = df["Volume"].rolling(10).mean()
df["vol_sma_20"] = df["Volume"].rolling(20).mean()
df["vol_ratio"] = df["Volume"] / df["vol_sma_20"]

df.dropna(inplace=True)
df.head()

Price,Close,High,Low,Open,Volume,return_1d,return_3d,return_5d,return_10d,sma_10,...,bb_width,rsi_14,stoch_k,adx,macd,macd_signal,macd_diff,vol_sma_10,vol_sma_20,vol_ratio
Date,,,,,,,,,,,,,,,,,,,,,
2020-03-13,151.218002,154.150394,133.985447,140.430996,92727400,0.142169,-0.012988,-0.016959,-0.019629,152.212944,...,0.313249,46.398590,55.601194,36.269676,-5.294389,-3.175327,-2.119062,69125970.0,62020600.0,1.495106
2020-03-16,128.929947,142.192354,128.530077,133.290451,87905900,-0.147390,-0.118532,-0.100916,-0.216274,148.655038,...,0.401757,36.896030,1.049995,36.139094,-6.741321,-3.888526,-2.852795,70813480.0,65258420.0,1.347043
2020-03-17,139.545593,140.431016,128.530082,133.290456,81059800,0.082337,0.054005,-0.089175,-0.109051,146.947015,...,0.360471,42.894903,28.925009,36.017840,-6.951301,-4.501081,-2.450220,71751760.0,67921800.0,1.193428
2020-03-18,133.671295,139.002920,128.549143,131.386321,81593200,-0.042096,-0.116036,-0.086116,-0.176781,144.076511,...,0.366090,40.595149,13.500028,35.905247,-7.505202,-5.101905,-2.403297,74929640.0,70501585.0,1.157324
2020-03-19,135.870575,142.953998,132.338372,135.927697,85922700,0.016453,0.053833,0.026248,-0.141697,141.833423,...,0.341798,41.852102,19.274990,35.086163,-7.678201,-5.617164,-2.061037,78740180.0,72954600.0,1.177756


In [10]:
df["future_return_3d"] = df["Close"].pct_change(3).shift(-3)

RET_TH = 0.002
df["target"] = (df["future_return_3d"] > RET_TH).astype(int)
df.dropna(inplace=True)

In [11]:
test_start = "2024-01-01"
test_end = "2024-05-06"

feature_cols = [
    "return_1d", "return_3d", "return_5d", "return_10d",
    "sma_10", "sma_20", "sma_50", "ema_20",
    "atr_14", "bb_width",
    "rsi_14", "stoch_k", "adx",
    "macd", "macd_signal", "macd_diff",
    "vol_ratio"
]

X = df[feature_cols]
y = df["target"]

train_mask = df.index < test_start
test_mask = (df.index >= test_start) & (df.index <= test_end)

X_train, y_train = X[train_mask], y[train_mask]
X_test, y_test = X[test_mask], y[test_mask]

In [14]:
tscv = TimeSeriesSplit(n_splits=5)

pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", HistGradientBoostingClassifier(random_state=42))
])

param_distributions = {
    "clf__max_depth": randint(2, 8),
    "clf__learning_rate": uniform(0.01, 0.2),
    "clf__max_iter": randint(100, 500),
    "clf__min_samples_leaf": randint(10, 60),
    "clf__l2_regularization": uniform(0.0, 1.0)
}

search = RandomizedSearchCV(
    pipe,
    param_distributions=param_distributions,
    n_iter=40,
    scoring="roc_auc",
    cv=tscv,
    n_jobs=-1,
    random_state=42,
    verbose=1
)

search.fit(X_train, y_train)
best_model = search.best_estimator_

print("Najlepsze parametry:", search.best_params_)
print("Test ROC-AUC:", search.best_score_)

df["proba"] = best_model.predict_proba(X)[:, 1]

print(classification_report(y_test, (df.loc[test_mask, "proba"] > 0.5).astype(int)))

Fitting 5 folds for each of 40 candidates, totalling 200 fits
Najlepsze parametry: {'clf__l2_regularization': np.float64(0.5026790232288615), 'clf__learning_rate': np.float64(0.02029575024999787), 'clf__max_depth': 6, 'clf__max_iter': 323, 'clf__min_samples_leaf': 42}
Test ROC-AUC: 0.5597075185640736
              precision    recall  f1-score   support

           0       0.45      0.56      0.50        36
           1       0.59      0.49      0.53        47

    accuracy                           0.52        83
   macro avg       0.52      0.52      0.52        83
weighted avg       0.53      0.52      0.52        83



In [15]:
from backtesting import Backtest


bt_df_full = df.copy()
bt_df_full["signal"] = 0

bt_data_test = bt_df_full.loc[test_start:test_end, ["Open","High","Low","Close","Volume","proba"]].copy()

In [16]:
class MLStrategyTP_SL(Strategy):
    proba_entry = 0.55
    sl_atr = 1.8
    tp_atr = 3.0

    def init(self):
        pass

    def next(self):
        proba = self.data.proba[-1]

        atr = df.loc[self.data.index[-1], "atr_14"]

        if not self.position:
            if proba > self.proba_entry:
                sl = self.data.Close[-1] - self.sl_atr * atr
                tp = self.data.Close[-1] + self.tp_atr * atr
                self.buy(sl=sl, tp=tp)

        else:
            if proba < 0.45:
                self.position.close()


def run_bt_for_threshold(th):
    MLStrategyTP_SL.proba_entry = th
    bt = Backtest(
        bt_data_test,
        MLStrategyTP_SL,
        cash=10000,
        commission=0.001,
        exclusive_orders=True
    )
    stats = bt.run()
    return stats

thresholds = np.arange(0.50, 0.71, 0.02)
results = []

for th in thresholds:
    s = run_bt_for_threshold(th)
    results.append((th, s["Return [%]"], s["Sharpe Ratio"], s["# Trades"], s["Max. Drawdown [%]"]))

res_df = pd.DataFrame(results, columns=["threshold","Return%","Sharpe","#Trades","MaxDD%"])
print(res_df.sort_values("Return%", ascending=False).head(10))

best_th = res_df.sort_values("Return%", ascending=False).iloc[0]["threshold"]
print("Najlepszy próg:", best_th)

Backtest.run:   0%|          | 0/82 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/82 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/82 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/82 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/82 [00:00<?, ?bar/s]

...

Backtest.run:   0%|          | 0/82 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/82 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/82 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/82 [00:00<?, ?bar/s]

    threshold   Return%    Sharpe  #Trades    MaxDD%
3        0.56  8.655728  1.722401        9 -5.352826
1        0.52  7.461676  1.515512        9 -5.263174
0        0.50  7.461676  1.515512        9 -5.263174
2        0.54  7.461676  1.515512        9 -5.263174
10       0.70  6.585663  1.756702        6 -3.138387
9        0.68  6.585663  1.756702        6 -3.138387
8        0.66  5.166152  1.312352        6 -4.428393
7        0.64  5.166152  1.312352        6 -4.428393
5        0.60  5.145150  1.193928        8 -4.035296
4        0.58  4.850882  1.059152        9 -5.342207
Najlepszy próg: 0.56


In [17]:
MLStrategyTP_SL.proba_entry = float(best_th)

bt_final = Backtest(
    bt_data_test,
    MLStrategyTP_SL,
    cash=10000,
    commission=0.001,
    exclusive_orders=True
)

stats_final = bt_final.run()
print(stats_final)
bt_final.plot()

Backtest.run:   0%|          | 0/82 [00:00<?, ?bar/s]

Start                     2024-01-02 00:00:00
End                       2024-04-30 00:00:00
Duration                    119 days 00:00:00
Exposure Time [%]                    59.03614
Equity Final [$]                  10865.57278
Equity Peak [$]                   11464.31289
Commissions [$]                     187.21154
Return [%]                            8.65573
Buy & Hold Return [%]                  5.1716
Return (Ann.) [%]                    28.66517
Volatility (Ann.) [%]                16.64257
CAGR [%]                             19.21935
Sharpe Ratio                           1.7224
Sortino Ratio                         3.85202
Calmar Ratio                          5.35515
Alpha [%]                             6.39305
Beta                                  0.43752
Max. Drawdown [%]                    -5.35283
Avg. Drawdown [%]                    -1.31909
Max. Drawdown Duration       19 days 00:00:00
Avg. Drawdown Duration        8 days 00:00:00
# Trades                          

GridPlot(id='p1329', ...)

Utworzony model przewiduje, czy 3-dniowa stopa zwrotu akcji MSFT przekroczy 0,2%. Model trenowano na danych 2020–2023 z wykorzystaniem wskaźników technicznych (m.in. RSI, MACD, ATR, Bollinger Bands, ADX, średnie kroczące) oraz optymalizacji hiperparametrów metodą RandomizedSearchCV z walidacją TimeSeriesSplit.
Uzyskany wynik ROC-AUC ≈ 0,56 wskazuje na niewielką, ale dodatnią przewagę predykcyjną nad losowym zgadywaniem. Dokładność ok. 52% i umiarkowane wartości precision/recall pokazują, że przewidywanie kierunku krótkoterminowych ruchów cen jest trudne i obarczone szumem rynkowym. W strategii handlowej wykorzystano prawdopodobieństwo predykcji (proba) oraz zarządzanie ryzykiem SL/TP w oparciu o ATR. Dodatkowo zoptymalizowano próg wejścia – najlepszy był 0,56. Zwrot strategii wyniósł 8,66% i był lepszy od strategii Buy&Hold (5,17%).